In [1]:
from ultralytics import YOLO
model = YOLO("yolo-Weights/yolov8n.pt")

In [2]:
import cv2
import math
import time

capture = cv2.VideoCapture(1)
capture.set(3,640)
capture.set(4,480)

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

target_class = "cell phone"
threshold = 0.5


while True:

    success, image = capture.read()
    if not success:
        break

    results = model(image, stream=True, verbose=False)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)


            ## Calculate centre coordinates of box
            x_centre = (x1+x2)//2
            y_centre = (y1+y2)//2
            image_width = image.shape[1]
            image_height = image.shape[0]
            norm_x = x_centre / image_width
            norm_y = y_centre / image_height

            confidence = float(box.conf[0])
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            if confidence >= threshold and detected_class == target_class:

                cv2.rectangle(image, (x1,y1),(x2,y2), (255,0,255),3)
                cv2.putText(image, f"{detected_class} {confidence:.2f}",(x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)

                cv2.circle(image, (x_centre,y_centre),radius=5,color=(0,0,255),thickness=-1)
                coord_text = f"({norm_x:.2f}, {norm_y:.2f})"
                cv2.putText(image, coord_text, (x_centre+10, y_centre),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)



    cv2.imshow('Webcam',image)

    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [3]:
import json
import requests

## Load API keys
with open("alpaca_keys.json","r") as file:
    data = json.load(file)

alpaca_key = data['alpaca_key']
alpaca_secret = data['alpaca_secret']


url = "https://paper-api.alpaca.markets/v2/orders"

headers = {
    "Apca-Api-Key-Id": alpaca_key,
    "Apca-Api-Secret-Key": alpaca_secret,
    "Content-Type": "application/json",
}

payload = {
    "symbol": "BTC/USD",
    "qty": "0.00009975",
    "side": "sell",
    "type": "market",
    "time_in_force": "gtc"      # good till cancelled
}

response = requests.post(url,json=payload,headers=headers)

print(response.status_code, response.json())

200 {'id': 'b1a83752-3701-4d17-aca8-649f48f6d1e4', 'client_order_id': 'e6beeec1-7e56-4902-a990-56649a67c435', 'created_at': '2025-01-07T19:13:14.024766037Z', 'updated_at': '2025-01-07T19:13:14.025827437Z', 'submitted_at': '2025-01-07T19:13:14.024766037Z', 'filled_at': None, 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340', 'symbol': 'BTC/USD', 'asset_class': 'crypto', 'notional': None, 'qty': '0.00009975', 'filled_qty': '0', 'filled_avg_price': None, 'order_class': '', 'order_type': 'market', 'type': 'market', 'side': 'sell', 'position_intent': 'sell_to_close', 'time_in_force': 'gtc', 'limit_price': None, 'stop_price': None, 'status': 'pending_new', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'subtag': None, 'source': None, 'expires_at': '2025-04-07T20:00:00Z'}
